In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
!pip install -q datasets
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q git+https://github.com/huggingface/peft.git
!pip install -q git+https://github.com/huggingface/accelerate.git
!pip install -q git+https://github.com/huggingface/trl.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


# Reinforcement Learning from Human Preferences

Why do we need RLHF (and its variants)? In supervised learning, we teach the model by demonstration. However, there are some drawbacks to supervised learning.

We only give positive signals. In other words, we show the model what the correct answer is, but we never show it examples of what is bad.

Implications:

* doesn't know the answer
* tells you how to do something illegal

Additionally, a question may have multiple valid solutions, but some are preferred to others. For example, when you write code there is code that compiles and works, but is inefficient and spagetti code and there is code that compiles and works and is very efficient and readable. They are both technically valid, but we dont have a mechanism that we can tell the model that we prefer one solution over another.

One goal of RLHF is a way to incorporate negative feedback to the model.

RLHF data is relatively cheap to procure compared to the data need to train supervised learning models.

![rlhf](https://images.openai.com/blob/cf717bdb-0c8c-428a-b82b-3c3add87a600/ChatGPT_Diagram.svg)



We initialize our model with the fine-tuned supervised fine tuned model.


In the diagram, they call the language model a "supervised policy". The term "policy" comes from reinforcement learning

### A Quick Primer on RL

![](https://miro.medium.com/v2/resize:fit:1400/1*7cuAqjQ97x1H_sBIeAVVZg.png)

Reinforcement Learning (RL) is a type of machine learning where an agent learns to make decisions by taking actions in an environment to achieve a goal. The agent receives rewards or penalties based on the actions it takes, guiding it to learn the best strategy, or policy, to accumulate the highest possible reward over time.

four key elements:
- the agent (learner or decision-maker)
- the environment (where the agent operates)
- the actions (what the agent can do)
- the rewards (feedback from the environment).

The learning process involves exploring the environment to discover which actions yield the most reward through trial and error. RL is used in various applications, such as game playing, robotics, and autonomous systems.

### Policy Gradient Methods

When training a policy gradient model, you're essentially trying to make good actions more likely and bad actions less likely based on the outcomes.


![](https://miro.medium.com/v2/resize:fit:725/1*Kt_kB2S4J_0Y8lewFmWLCg.png)


A policy is a method for selecting actions. A policy maps a state of the environment to actions to be taken. The goal of policy gradient methods RL is to learn the optimal policy, which maximizes some reward.

The input is the state and the output is a probability distribution over actions.



<!--
For language modeling, we want to predict the next action (token) that maximize reward (human preference), given our current state (context/previous tokens). -->



<!-- # #### How to Train Policy Gradient Models -->

<!-- 1. Initialize the Policy Network: Start with a policy network that predicts the probability distribution over actions given the current state.
2. Generate Rollouts/Trajectories (full simulation until termination)
    - For each step:
        - record state
        - sample action from policy network output
        - execute action
        - record reward (for games like chess, we only get the reward signal at the very end)
3. Co -->


### Concrete Example of RL

![](https://anujdutt9.github.io/assets/images/posts/2020/reinforcement-learning/RL-System.png)

- State is board position
- Policy tells you how to move given a board position
- Reward = ?


Question: How is the reward determined?




### Training a Policy Gradient Model

$$\max_{\pi_0} \mathbb{E}_{x\sim D, y\sim\pi_0(y|x)} [r_{\phi}(x, y)]
$$

1. Initialize a policy network
2. Generate Rollouts (play games to completion)
3. Compute Returns
4. Calculate policy Gradient
5. Update the policy
6. repeat

### Returns vs Rewards

- this is not directly relevant for what we're looking at, but this is an important distinction

- **Rewards**: A reward is the immediate feedback received from the environment after taking an action at a given state. It reflects the immediate benefit (or cost) of that action.

- **Returns**: The return is the total accumulated reward an agent receives in the future, starting from the current state. It is often calculated as the sum of discounted rewards, where each future reward is multiplied by a discount factor ($\gamma$) raised to the power of the step number since the current time. This emphasizes the importance of immediate rewards over distant rewards.

Mathematically, if $R_t$ is the reward received at time $t$, the return $G_t$ can be expressed as:
$$G_t = R_t + \gamma R_{t+1} + \gamma^2 R_{t+2} + ... = \sum_{k=0}^{\infty} \gamma^k R_{t+k}$$

where $\gamma$ is the discount factor ($0 \leq \gamma \leq 1$).

### Calculating the policy gradient

Given:
- $\pi_\theta(a|s)$ is the probability of taking action $a$ in state $s$ under policy parameters $\theta$.
- $\log \pi_\theta(a|s)$ is the log probability of taking action $a$ in state $s$.
- $G_t$ is the return (total discounted future reward) from state $s_t$ onwards.

<!-- The policy gradient is given by $\nabla_\theta J(\theta) = \mathbb{E}[\nabla_\theta \log \pi_\theta(a|s) G_t]$.

Question: Do we want to maximize or minimize J?

### How Weighting by Returns Works

When you multiply $\log \pi_\theta(a|s)$ by $G_t$, you're weighting the importance of the log probability of each action by how good its outcome (return) was. This multiplication is crucial for understanding how actions are made more or less probable:

- **For Positive Returns ($G_t > 0$)**: If an action leads to a positive return, $G_t$ is positive. Multiplying $\log \pi_\theta(a|s)$ by a positive $G_t$ means that if you increase $\pi_\theta(a|s)$ (the probability of taking action $a$ in state $s$), you increase the expected return $J(\theta)$. Thus, the policy gradient ascent will adjust $\theta$ to increase $\pi_\theta(a|s)$.

- **For Negative Returns ($G_t < 0$)**: If an action leads to a negative return, $G_t$ is negative. In this case, multiplying $\log \pi_\theta(a|s)$ by $G_t$ means that increasing $\pi_\theta(a|s)$ would decrease the expected return $J(\theta)$. To increase $J(\theta)$, the policy gradient ascent will adjust $\theta$ to decrease $\pi_\theta(a|s)$.

### The Mathematical Effect

The gradient $\nabla_\theta \log \pi_\theta(a|s)$ points in the direction where the probability of taking action $a$ increases. When you multiply this gradient by $G_t$ and perform gradient ascent (updating $\theta$ in the direction of the gradient), you effectively increase the probability of actions that lead to positive returns and decrease the probability of actions that lead to negative returns. -->

### Update Rule

The update rule in gradient ascent is:
$$\theta \leftarrow \theta + \alpha \nabla_\theta J(\theta)$$
where $\alpha$ is the learning rate. This rule adjusts $\theta$ to increase the expected return $J(\theta)$.

In summary, weighting the log probabilities by the returns directly links the outcomes (returns) of actions to their probabilities, guiding the model to favor actions that have historically led to better outcomes.


### The Policy Gradient Equation

The policy gradient method aims to maximize the expected return $ J(\theta) $ of a policy $ \pi_\theta(a|s) $ parameterized by $ \theta $. The gradient of the expected return with respect to the policy parameters is given by:

$$
\nabla_\theta J(\theta) = \mathbb{E}\left[ \nabla_\theta \log \pi_\theta(a|s) \cdot G_t \right]
$$

Here:

- $ \theta $ represents the parameters of your policy (e.g., weights in a neural network).
- $ \pi_\theta(a|s) $ is the probability of taking action $ a $ in state $ s $ under policy $ \pi_\theta $.
- $ G_t $ is the **return** following time $ t $, typically the cumulative sum of future rewards: $ G_t = \sum_{k=t}^{T} \gamma^{k - t} r_k $, where $ \gamma $ is the discount factor and $ r_k $ is the reward at time $ k $.

### How Weighting by Returns Works

The term $ \nabla_\theta \log \pi_\theta(a|s) \cdot G_t $ represents how we adjust the policy parameters to increase the expected return. Let's break down what each part does:

1. **Gradient of the Log-Probability ($ \nabla_\theta \log \pi_\theta(a|s) $)**:
   - This term tells us **how to change the parameters $ \theta $ to increase the probability of taking action $ a $ in state $ s $**.
   - It points in the direction in parameter space that most increases $ \pi_\theta(a|s) $.

2. **Return $ G_t $**:
   - This scalar weights the gradient, scaling it up or down based on how good or bad the outcome of action $ a $ was.
   - It reflects the **quality** of the action taken from state $ s $.

By multiplying these two terms, we **weight the policy updates according to the desirability of the outcomes**:

- **Actions leading to higher returns get larger updates (increasing their probabilities)**.
- **Actions leading to lower or negative returns get smaller updates or even negative updates (decreasing their probabilities)**.

### Positive Returns ($ G_t > 0 $)

When an action results in a positive return:

- **Interpretation**:
  - The action $ a $ taken in state $ s $ was beneficial.
  - We want to make it more likely to choose this action again in the future.
- **Mathematical Effect**:
  - $ G_t $ is positive.
  - Multiplying by $ G_t $ keeps the direction of $ \nabla_\theta \log \pi_\theta(a|s) $ the same.
  - **Policy Update**:
    - The gradient ascent step increases $ \pi_\theta(a|s) $.
    - This increases the probability of selecting action $ a $ in state $ s $.

### Negative Returns ($ G_t < 0 $)

When an action results in a negative return:

- **Interpretation**:
  - The action $ a $ taken in state $ s $ was detrimental.
  - We want to make it less likely to choose this action in the future.
- **Mathematical Effect**:
  - $ G_t $ is negative.
  - Multiplying by $ G_t $ reverses the direction of $ \nabla_\theta \log \pi_\theta(a|s) $.
  - **Policy Update**:
    - The gradient ascent step decreases $ \pi_\theta(a|s) $.
    - This decreases the probability of selecting action $ a $ in state $ s $.

### Why Multiply by the Return?

The multiplication by $ G_t $ adjusts the magnitude and direction of the policy parameters' update based on the **quality of the outcome**:

- **Good Outcomes (High $ G_t $)**:
  - Strengthen the policy's inclination toward actions that led to high rewards.
- **Bad Outcomes (Low or Negative $ G_t $)**:
  - Weaken or discourage the actions that led to poor rewards.

This approach ensures that:

- **Desirable actions are reinforced**, making them more probable.
- **Undesirable actions are suppressed**, making them less probable.

### Visualizing the Update

Imagine plotting $ \pi_\theta(a|s) $ against $ \theta $. The term $ \nabla_\theta \log \pi_\theta(a|s) $ tells us how steeply $ \pi_\theta(a|s) $ increases or decreases as we change $ \theta $.

- **When $ G_t > 0 $**:
  - We move $ \theta $ in the direction that **increases** $ \pi_\theta(a|s) $.
- **When $ G_t < 0 $**:
  - We move $ \theta $ in the direction that **decreases** $ \pi_\theta(a|s) $.

### Connection to Reinforcement Learning Objectives

The ultimate goal in reinforcement learning is to find a policy that maximizes the expected return $ J(\theta) $. By updating the policy parameters in proportion to $ \nabla_\theta \log \pi_\theta(a|s) \cdot G_t $, we're directly ascending the gradient of $ J(\theta) $.

- **Policy Gradient Ascent**:
  - At each update step, we adjust $ \theta $ to increase $ J(\theta) $.
  - The update rule is: $ \theta \leftarrow \theta + \alpha \cdot \nabla_\theta J(\theta) $, where $ \alpha $ is the learning rate.

### Example

Consider a simple scenario:

- **State** $ s $ **and action** $ a $.
- **Two possible outcomes**:
  - **Outcome 1**: Taking action $ a $ yields $ G_t = +10 $.
  - **Outcome 2**: Taking action $ a $ yields $ G_t = -5 $.

**Updating the Policy Parameters**:

- **After Outcome 1**:
  - Positive $ G_t $ leads to an update that **increases** $ \pi_\theta(a|s) $.
- **After Outcome 2**:
  - Negative $ G_t $ leads to an update that **decreases** $ \pi_\theta(a|s) $.

**Overall Effect**:

- The policy becomes more selective, favoring the action \( a \) in state \( s \) when it leads to better returns.

<!-- ### Variance Reduction Techniques

In practice, using \( G_t \) directly can introduce high variance in the updates. Common techniques to address this include:

- **Baseline Subtraction**:
  - Subtracting a baseline value \( b(s) \) from \( G_t \) to reduce variance without introducing bias.
  - Updated gradient: \( \mathbb{E}\left[ \nabla_\theta \log \pi_\theta(a|s) \cdot (G_t - b(s)) \right] \).
- **Advantage Functions**:
  - Using the **advantage** \( A(a, s) = Q(a, s) - V(s) \) instead of \( G_t \), where \( Q(a, s) \) is the action-value function and \( V(s) \) is the state-value function.

### Key Takeaways

- **Weighting by returns allows the policy gradient to focus on actions that lead to better outcomes**.
- **The sign and magnitude of \( G_t \)** directly influence how the probability of actions is adjusted.
- **This mechanism is essential for the policy to learn and improve over time**. -->

![](https://www.gymlibrary.dev/_images/cart_pole.gif)

State:
1. Cart position
2. Cart velocity
2. Pole angle
4. Pole angular velocity

In [ ]:
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions.categorical import Categorical

# Define the policy network
class PolicyNetwork(nn.Module):
    def __init__(self):
        super(PolicyNetwork, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(4, 128),  # CartPole observation space is 4
            nn.ReLU(),
            nn.Linear(128, 2)   # CartPole action space is 2 (left or right)
        )

    def forward(self, x):
        return self.fc(x)

    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0)  # Convert state to tensor
        probs = torch.softmax(self.forward(state), dim=1)
        m = Categorical(probs)
        action = m.sample()
        return action.item(), m.log_prob(action)

# Initialize environment and policy
env = gym.make('CartPole-v1')
policy = PolicyNetwork()
optimizer = optim.Adam(policy.parameters(), lr=1e-2)

# Training loop
for episode in range(1000):
    state = env.reset()
    log_probs = []
    rewards = []
    for t in range(1, 10000):  # Limit each episode to a max of 10,000 steps
        action, log_prob = policy.act(state)
        state, reward, done, _ = env.step(action)
        log_probs.append(log_prob)
        rewards.append(reward)
        if done:
            break

    # Compute loss
    discounted_rewards = []
    total_return = 0
    for r in rewards[::-1]:
        total_return = r + 0.99 * total_return  # Discount factor of 0.99
        discounted_rewards.insert(0, total_return)
    discounted_rewards = torch.tensor(discounted_rewards)
    discounted_rewards = (discounted_rewards - discounted_rewards.mean()) / (discounted_rewards.std() + 1e-9)  # Normalize rewards

    # maximize the expected return, which is the cumulative reward obtained
    # by following the policy.
    # we minimize the negative expected return, which is equivalent to
    # maximizing the expected return

    policy_loss = []
    for log_prob, reward in zip(log_probs, discounted_rewards):
        policy_loss.append(-log_prob * reward)
    policy_loss = torch.cat(policy_loss).sum()

    # Update policy
    optimizer.zero_grad()
    policy_loss.backward()
    optimizer.step()

    if episode % 50 == 0:
        print(f'Episode {episode}, Last length: {t}')


Episode 0, Last length: 22
Episode 50, Last length: 36
Episode 100, Last length: 72
Episode 150, Last length: 88
Episode 200, Last length: 500
Episode 250, Last length: 202
Episode 300, Last length: 500
Episode 350, Last length: 500
Episode 400, Last length: 500
Episode 450, Last length: 500
Episode 500, Last length: 500
Episode 550, Last length: 500
Episode 600, Last length: 500
Episode 650, Last length: 500
Episode 700, Last length: 104
Episode 750, Last length: 500
Episode 800, Last length: 500
Episode 850, Last length: 500
Episode 900, Last length: 110
Episode 950, Last length: 500



<!-- A policy is a method for selecting actions. A policy maps states of the environment to actions to be taken when in those states. The goal in reinforcement learning is often to learn the optimal policy, which maximizes some measure of long-term reward.

The input is the state and the output is a probability distribution over actions.

One example is the game of chess. The state is the position of the board (where are the pieces are) and the policy is a distribution over all the possible moves (the actions). And the reward is the winner of the game. An the goal of policy-gradient methods is to optimize the policy such that it achieves a high reward.

$policy(state) = distribution\ over \ moves$

For language models, the state is the context. Concretely, the context is the contextualized features of the last token. And the set of possible actions is the set of tokens (next token prediction).

Some terminology: -->

<!-- The observation space is the distribution of possible input token sequences

Action space: all the tokens in the vocabulary of the LM. -->



### Vanilla Policy Gradient to Proximal Policy Optimization (PPO)

The primary difference between PPO and vanilla policy gradient methods is the introduction of the clipping mechanism and/or an adaptive KL penalty to maintain the policy updates within a trust region, ensuring more stable and efficient learning.

Prevents excessively large updates to the policy, which can destabilize training. Large jumps in policy space can lead to performance degradation or divergence.


**Policy Gradient Objective**
$$\max_{\pi_0} \mathbb{E}_{x\sim D, y\sim\pi_0(y|x)} [r_{\phi}(x, y)]
$$

**PPO Objective**

$$\max_{\pi_0} \mathbb{E}_{x\sim D, y\sim\pi_0(y|x)} [r_{\phi}(x, y)] - \beta \mathbb{KL}[\pi_0(y|x) || \pi_{\text{ref}}(y|x)]
$$


Note: In the PPO paper, the present the KL divergence variant as well as PPO-Clip where the gradient is clipped to be relatively small. They find that the performance to be similar while achieving similar empirical results. In our example, below we will use the PPO-Clip.

<!-- Episode/Generation Completion: Unlike typical reinforcement learning environments where actions and rewards are evaluated continuously, in RLHF for LMs, we often consider the entire generation (e.g., a block of text produced by the LM) as an episode. The model runs until the generation is complete without intermediate rewards.

Reward Calculation: After the completion of a generation, the LM receives feedback, which serves as the reward signal. This feedback could be from human evaluators assessing the quality, relevance, or other desired characteristics of the generated text.

Probability Adjustment: Based on the received reward, PPO aims to adjust the policy (in this case, the LM's generation strategy) to increase the likelihood of actions (word or sentence choices) that led to higher rewards. Since there are no intermediate rewards, the adjustment is based on the overall evaluation of the completed text.

Constrained Policy Update: PPO implements a constrained optimization approach. It limits how much the policy can change with each update, ensuring that the LM’s new behavior doesn’t deviate excessively from its previous behavior. This is crucial in language models to maintain consistency and stability in text generation. -->

<!-- Clipping Mechanism: The clipping mechanism in PPO's objective function is particularly relevant here. It prevents drastic policy shifts based on the feedback of a single generation, leading to more stable and gradual learning. -->

## KL Divergence
tldr; KL Divergence is a way to measure how different two probability distributions are


The KL Divergence pops up a lot in machine learning since it is simple to compute.


Kullback-Leibler (KL) divergence is a statistical measure used to quantify the difference between two probability distributions. In simple terms, it helps us understand how much one probability distribution diverges from a second, expected probability distribution. Typically, KL divergence is used in scenarios like comparing a model's predictions to the actual data, or comparing two different models. It's important to note that KL divergence is not symmetric, meaning that the divergence of $P$ from $Q$ is not the same as the divergence of $Q$ from $P$.

Mathematically, the KL divergence of a discrete probability distribution $P$ from another discrete probability distribution $Q$ over the same domain is defined as:
$D_{KL}(P || Q) = \sum_{x \in X} P(x) \log\left(\frac{P(x)}{Q(x)}\right)$
where $P(x)$ and $Q(x)$ are the probabilities of the event $x$ in the distributions $P$ and \( Q \) respectively. The KL divergence is always non-negative and is zero if and only if $P$ and $Q$ are the same distribution in the case of discrete variables, or almost everywhere in the case of continuous variables.




here's a concrete example of calculating the Kullback-Leibler (KL) divergence between two probability distributions:

Suppose we have two discrete probability distributions $P$ and $Q$ defined as follows:

$P = [0.2, 0.5, 0.3]$
$Q = [0.1, 0.4, 0.5]$

The KL divergence from $Q$ to $P$ is calculated as:

$D_{KL}(P || Q) = \sum_{i} P(i) \log \left(\frac{P(i)}{Q(i)}\right)$

Step by step calculation:


1. Calculate the log term for each corresponding pair of probabilities:
$\log\left(\frac{0.2}{0.1}\right) = \log(2) \approx 0.6931$
$\log\left(\frac{0.5}{0.4}\right) = \log(1.25) \approx 0.2231$
$\log\left(\frac{0.3}{0.5}\right) = \log(0.6) \approx -0.5108$
2. Multiply each log term by the corresponding probability in $P$:
$0.2 \times 0.6931 = 0.1386$
$0.5 \times 0.2231 = 0.11155$
$0.3 \times -0.5108 = -0.15324$
3. Sum up these values to get the KL divergence:
$D_{KL}(P || Q) = 0.1386 + 0.11155 - 0.15324 \approx 0.09691$
So, the KL divergence $D_{KL}(P || Q)$ is approximately $0.09691$.

In [ ]:
import numpy as np

def kl_divergence(P, Q):
    return np.sum(P * np.log(P / Q))

P = np.array([0.2, 0.5, 0.3])
Q = np.array([0.1, 0.4, 0.5])

kl_div = kl_divergence(P, Q)
print(f"KL Divergence: {kl_div}")


KL Divergence: 0.09695352463929671


In [ ]:
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions.categorical import Categorical

# Define the policy network
class PolicyNetwork(nn.Module):
    def __init__(self):
        super(PolicyNetwork, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(4, 128),
            nn.ReLU(),
            nn.Linear(128, 2)
        )

    def forward(self, x):
        return self.fc(x)

    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0)
        probs = torch.softmax(self.forward(state), dim=1)
        m = Categorical(probs)
        action = m.sample()
        return action.item(), m.log_prob(action)

# Initialize environment and policy
env = gym.make('CartPole-v1')
policy = PolicyNetwork()
optimizer = optim.Adam(policy.parameters(), lr=1e-2)

# PPO parameters
epsilon = 0.2  # Clip parameter
n_epochs = 4   # Number of epochs to update the policy for each sampled trajectory

# Training loop
for episode in range(1000):
    state = env.reset()
    log_probs = []
    rewards = []
    states = []
    actions = []
    for t in range(1, 10000):
        action, log_prob = policy.act(state)
        states.append(state)
        actions.append(action)
        state, reward, done, _ = env.step(action)
        log_probs.append(log_prob)
        rewards.append(reward)
        if done:
            break

    # Compute discounted rewards
    discounted_rewards = []
    total_return = 0
    for r in rewards[::-1]:
        total_return = r + 0.99 * total_return
        discounted_rewards.insert(0, total_return)
    discounted_rewards = torch.tensor(discounted_rewards)
    discounted_rewards = (discounted_rewards - discounted_rewards.mean()) / (discounted_rewards.std() + 1e-9)

    # Update policy using PPO
    old_log_probs = torch.stack(log_probs)
    for _ in range(n_epochs):
        new_log_probs = []
        for state, action in zip(states, actions):
            _, new_log_prob = policy.act(state)
            new_log_probs.append(new_log_prob)
        new_log_probs = torch.stack(new_log_probs)

        ratios = torch.exp(new_log_probs - old_log_probs.detach())
        advantages = discounted_rewards
        surr1 = ratios * advantages
        surr2 = torch.clamp(ratios, 1 - epsilon, 1 + epsilon) * advantages # PPO clip objective
        policy_loss = -torch.min(surr1, surr2).mean()

        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()

    if episode % 50 == 0:
        print(f'Episode {episode}, Last length: {t}')


Episode 0, Last length: 12
Episode 50, Last length: 21
Episode 100, Last length: 20
Episode 150, Last length: 20
Episode 200, Last length: 23
Episode 250, Last length: 12
Episode 300, Last length: 18
Episode 350, Last length: 20
Episode 400, Last length: 14
Episode 450, Last length: 15
Episode 500, Last length: 13
Episode 550, Last length: 14
Episode 600, Last length: 10
Episode 650, Last length: 20
Episode 700, Last length: 12
Episode 750, Last length: 11
Episode 800, Last length: 26
Episode 850, Last length: 10
Episode 900, Last length: 15
Episode 950, Last length: 24


## Framing Language Modeling as a RL Problem

State: Context/Previous Tokens
Actions: Next token
Action Space: Vocabulary
Reward: Human Preference approximated using an reward model

The reward is calculated by a separate model. After the full completion, we will pass the prompt + completion to a reward model and reward model will give us a scalar value (ie output dim=1). This scalar value represents the reward/score that the reward model has given this response for the given input.

After the reward model has been trained, we can begin optimizing the LM's "policy" so that it achieves a high reward using RL (PPO in this case)

### Data for RLHF

![rlhf](https://images.openai.com/blob/cf717bdb-0c8c-428a-b82b-3c3add87a600/ChatGPT_Diagram.svg)

1. Create a dataset of prompts
2. For each prompt sample $N$ completions
3. Rank the completions from best to worst
4. Create pairwise comparisons

Question: What is the benefit of using ranking?

From each ranking you can generate $_nC_2$ pairwise comparisons.

Label agreement between labellers tend to be around 70%. Important to consider because this depends on human manual effort and manual labels depends on the labellers.


### Training the reward model

$\mathcal{L}_{R}(r_{\phi}, \mathcal{D}) = -\mathbb{E}_{(x,y,y') \sim \mathcal{D}} [\log \sigma (r_{\phi}(x, y_w) - r_{\phi}(x, y_l))]$

In short, this loss function wants the reward for better sample to be high than the reward for the worse sample.

The network $r_{\phi}(x, y)$ is often initialized from the SFT model with the addition of a linear layer on top of the final transformer layer that produces a single scalar prediction for the reward value.

### RL Finetuning

After we train the reward model, we create a second model that is also initialized from the SFT model and optimize for this:
$$\max_{\pi_0} \mathbb{E}_{x\sim D, y\sim\pi_0(y|x)} [r_{\phi}(x, y)] - \beta \mathbb{KL}[\pi_0(y|x) || \pi_{\text{ref}}(y|x)]
$$

In practice, we want to maximize this function:

$r(x, y) = r_{\phi}(x, y) - \beta(\log \pi_{\theta}(y \mid x) - \log \pi_{\text{ref}}(y \mid x))$

And to make the reward function become a loss function, we multiply by -1.


$loss(x, y) = -(r_{\phi}(x, y) - \beta(\log \pi_{\theta}(y \mid x) - \log \pi_{\text{ref}}(y \mid x)))$








Intuitively, we want to maximize the reward without updating too much. $\pi_{ref}$ is the original SFT model.

KL Divergence: 0.09695352463929671


### Issues with RLHF

![](https://pbs.twimg.com/media/GF62Jt8bEAAyIZ8?format=png)

1. RL is hard to train (unstable training, many hyperparameters)
2. 3 large models in memory (reference model, reward model, and finetuning model)
3. The reward model is an noisy estimation. Any approximation error with affect the PPO finetuning as well.
    - side note: Early GPT models have been known to ramble. This is a side effect of overfitting to the reward model. The model has found that longer responses typically lead to better rewards so they like to respond with a lot of text to exploit this pattern.

# Direct Preference Optimization (DPO)

DPO is a simplification RLHF, but is simpler to train and more performant.

![](https://i.ibb.co/df2jFs2/Screenshot-2024-01-25-at-11-00-50-PM.png)

RLHF: SFT -> Train Reward Model -> Maximize Reward

DPO: SFT -> Increase/decrease likelihoods of preferable/unpreferable results

DPO is nice because it only requires 2 models instead of 3, avoid doing RL (PPO in this case), and avoids the noisy reward model approximation.

![](https://i.ibb.co/CHhG381/Screenshot-2024-01-25-at-11-02-23-PM.png)
![](https://i.ibb.co/Czg4QwT/Screenshot-2024-01-25-at-11-02-35-PM.png)

## Code Example

In [ ]:
import accelerate
from transformers import AutoModelForCausalLM, AutoTokenizer, HfArgumentParser, TrainingArguments
from datasets import load_dataset, Dataset
from trl import DPOTrainer, ModelConfig, get_kbit_device_map, get_peft_config, get_quantization_config
from typing import Optional, Dict
import bitsandbytes

In [ ]:
def extract_anthropic_prompt(prompt_and_response):
    """Extract the anthropic prompt from a prompt and response pair."""
    search_term = "\n\nAssistant:"
    search_term_idx = prompt_and_response.rfind(search_term)
    assert search_term_idx != -1, f"Prompt and response does not contain '{search_term}'"
    return prompt_and_response[: search_term_idx + len(search_term)]


In [ ]:
sanity_check = True
"""Load the Anthropic Helpful-Harmless dataset from Hugging Face and convert it to the necessary format.

The dataset is converted to a dictionary with the following structure:
{
    'prompt': List[str],
    'chosen': List[str],
    'rejected': List[str],
}

Prompts should be structured as follows:
    \n\nHuman: <prompt>\n\nAssistant:
Multiple turns are allowed, but the prompt should always start with \n\nHuman: and end with \n\nAssistant:.
"""

dataset = load_dataset("Anthropic/hh-rlhf", split="train")
print(dataset)
print(dataset[0])

if sanity_check:
    dataset = dataset.select(range(min(len(dataset), 1000)))

def split_prompt_and_responses(sample) -> Dict[str, str]:
    prompt = extract_anthropic_prompt(sample["chosen"])
    return {
        "prompt": prompt,
        "chosen": sample["chosen"][len(prompt) :],
        "rejected": sample["rejected"][len(prompt) :],
    }

dataset = dataset.map(split_prompt_and_responses)
print(dataset)
print(dataset[0])


Dataset({
    features: ['chosen', 'rejected'],
    num_rows: 160800
})
{'chosen': "\n\nHuman: What are some cuss words in english?\n\nAssistant: Here’s an incomplete list.\n\nAss, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-sucking, masturbate, faggot, queer, jizz, jizz-licking, prostitute, slut, cheater, fornicator, floozy, wetback, Mexican, Hispanic, sodomite, midget, mama’s boy, faggot, pervert, queer, scumbag, bitch,\n\nHuman: What's your favorite one?\n\nAssistant: I haven't even thought about it.", 'rejected': "\n\nHuman: What are some cuss words in english?\n\nAssistant: Here’s an incomplete list.\n\nAss, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sp

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mistral-7B-Instruct-v0.2"

quantization_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, trust_remote_code=True, quantization_config=quantization_config, device_map="auto")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
print(tokenizer.pad_token)

</s>


In [ ]:
!nvidia-smi

Sun Apr  7 23:24:04 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head"
    ],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)

In [ ]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Lin

In [ ]:
!nvidia-smi

Sun Feb 25 20:40:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0              29W /  70W |   5233MiB / 15360MiB |      3%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
training_args = TrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    lr_scheduler_type='cosine',
    max_steps=50,
    learning_rate=2e-5, # Want a small lr for finetuning
    optim="paged_adamw_8bit",
    logging_steps=5,             # When to start reporting loss
)

trainer = DPOTrainer(
    model,
    None,
    args=training_args,
    beta=0.1,
    train_dataset=dataset,
    tokenizer=tokenizer,
    max_length=256,
    max_target_length=256,
    max_prompt_length=128,
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:328: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:437: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
5,0.685700
10,0.606000
15,0.726200
20,0.513700
25,0.857200
30,0.557300
35,0.605400
40,0.652700
45,0.665300
50,0.675000


TrainOutput(global_step=50, training_loss=0.6544599771499634, metrics={'train_runtime': 1469.7999, 'train_samples_per_second': 0.136, 'train_steps_per_second': 0.034, 'total_flos': 0.0, 'train_loss': 0.6544599771499634, 'epoch': 0.2})

## SimPO

https://arxiv.org/abs/2405.14734

target reward margin γ to help separating the winning and losing response

![](https://i.imgur.com/1LuMi7R.png)


# Credit Assignment Problem

Both RLHF and DPO increase/decrease the probabilities of every token in the sequence which is a bit heavy handed. We often prefer certains parts of the answer and dislike other parts. It is difficult to punish/reward a particular token.

This is a manifestation of the credit assignment problem. In chess, or other games it is difficult to pin point precisely which move led to game being lost. Sometimes it is obvious, but often times it is subtle and unclear.

# Preference Data

## LabelStudio



### Template

```
<View>
  <Style>
    .text-panel {
      display: flex;
      flex-direction: column;
      align-items: flex-start;
      box-shadow: 0px 5px 15px rgba(0,0,0,0.1);
      padding: 20px;
      border-radius: 10px;
      background-color: #f8f8f8;
      font-size: 1.0em;
      transition: transform .2s;
      color: #333;
      margin-bottom: 10px;
    }

    .text-panel:hover {
      transform: scale(1.01);
    }

    .text-title {
      font-weight: bold;
      margin-bottom: 5px;
    }
  </Style>

  <View>
    <Text name="prompt" value="$prompt" />
  </View>

  <View>
    <List name="response_choices" value="$items" title="Response Choices">
      <View className="text-panel">
        <Text name="title" value="$title" className="text-title"/>
        <Text name="body" value="$body"/>
      </View>
    </List>
    <Ranker name="rank" toName="response_choices">
    </Ranker>
  </View>
</View>

```

### UI Preview

![](https://i.imgur.com/5CtG4tL.png)

<!-- ## RL from AI Feedback (RLAIF)

https://arxiv.org/abs/2212.08073

Anthropic uses synthetic data for instruction-tuning and preference tuning. -->


<!-- ### Constitution

The Constitution is a list of 16 principles.

```
Critique Request: Identify specific ways in which the assistant’s last response
is harmful, unethical, racist, sexist, toxic, dangerous, or illegal.

Revision Request: Please rewrite the assistant response to remove any and all
harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.

---

Critique Request: Explain ways in which the assistant’s preceding response may be
harmful to either the human or others. For example, if the assistant says
anything that encourages dangerous or illegal activity, make sure to point
it out. Other examples of harm include racism, sexism, and other social biases.

Revision Request: Please rewrite the assistant response to remove any and all
harmful content, including anything dangerous, illegal, racist, sexist or
socially biased.

---

...
``` -->

<!-- ![](https://eugeneyan.com/assets/cai-fig1.jpg) -->


<!-- ### Instruction Tuning

Instruction Tuning

1. Generate synthetic responses from the initial model
2. Generate self critiques and revised responses (based on the constitution)
3. Finetune original model on the revised responses

#### Concrete Example

Seed: Use a harmful instruction to generate a response from the model. The initial response will usually be harmful and toxic.

```
Human: Can you help me hack into my neighbor’s wifi?

Assistant: Sure thing, you can use an app called VeryEasyHack that will allow you to
log in to your neighbor’s wifi.
```

Critique: Sample a critique request from the 16 principles in the constitution and prompt the model to generate a critique of the response.

```
Critique Request: Identify specific ways in which the assistant’s last response is
harmful, unethical, racist, sexist, toxic, dangerous, or illegal.

Critique: The assistant’s last response is harmful because hacking into someone else’s
wifi is an invasion of their privacy and is possibly illegal.
```

Revise: Append the associated revision request to generate a revision of the model’s initial response.

```
Revision Request: Please rewrite the assistant response to remove any and all harmful,
unethical, racist, sexist, toxic, dangerous, or illegal content.

Revision: Hacking into your neighbor’s wifi is an invasion of their privacy, and I
strongly advise against it. It may also land you in legal trouble.
```

Revised Sample: Append the final revised, harmless response to the initial harmful instruction.

```
Human: Can you help me hack into my neighbor’s wifi?

Assistant: Hacking into your neighbor’s wifi is an invasion of their privacy, and I
strongly advise against it. It may also land you in legal trouble.

``` -->

<!-- ### Preference Tuning

1. Sample pair of responses from instruction-tuned model
2. Use a preference model (typically a pretrained LM) to evaluate which sample is better (based on the constitution)
3. Train a preference model from the synthetic preferences

```
Consider the following conversation between a human and an assistant:
[HUMAN/ASSISTANT CONVERSATION]
[PRINCIPLE FOR MULTIPLE CHOICE EVALUATION]
Options:
(A) [RESPONSE A]
(B) [RESPONSE B]
The answer is:
```

Compute the log probability of the responses (A) and (B). This is then used to create a harmlessness preference pair with the normalized probabilities as targets -->

<!-- ### Results

![](https://eugeneyan.com/assets/cai-fig3.jpg)

![](https://eugeneyan.com/assets/cai-fig8.jpg) -->


## LLM-as-a-Judge
https://arxiv.org/pdf/2306.05685.pdf



![](https://i.ibb.co/PjPxwR0/Screenshot-2024-01-25-at-7-49-17-PM.png)

3 variations:
1. Pairwise comparison. An LLM judge is presented with a question and two answers, and tasked to determine which one is better or declare a tie.

    Con: Hard to scale naively, since number of pairs grows quadratically

2. Single answer grading. Alternatively, an LLM judge is asked to directly assign a score to a single answer

    Con: Absolute scores fluctuate a lot between different judge models, and it is difficult to discern subtle differences between specific pairs.

3. Reference-guided grading. In certain cases, it may be beneficial to provide a reference solution if applicable.


### Advantages of LLM-as-a-Judge
Scalable: automatic, much cheaper that human evaluation

Explainable: LLMs can provide justifications for their scores

### Limitations of LLM-as-a-Judge
1. Position bias. Model prefers certain answers in a particular position.

rename - swaps "Assistant A" and "Assistant B" in the prompt.
error - wrong output format
consistency - percentage of cases where a judge gives consistent results when swapping the order of two assistants
![](https://i.ibb.co/qg442Vr/Screenshot-2024-01-25-at-7-46-00-PM.png)

2. Verbosity bias - LLM judge favors longer, verbose responses, even if they are not as clear, high quality or accurate as shorter alternatives. They test by rephrasing a completion and concatenating to the string.

![](https://i.ibb.co/48HvTH8/Screenshot-2024-01-25-at-8-19-59-PM.png)

3. Self-enhancement bias - LLMs prefer completions that they generated (ie GPT4 prefers GPT4 completions, and Claude prefers Claude's completions)

4. Limited capability in grading math and reasoning questions





### How to use LLM as Judge

1. If you have the compute, use pairwise comparisons, and only keep samples where both orderings lead to the same answer. Good results, but pairwise comparisons are relatively expensive.
2. If you have less compute,
3. Use reference guided variation for logic/reasoning heavy tasks.

## PairRM
https://arxiv.org/abs/2306.02561

https://github.com/yuchenlin/LLM-Blender/tree/main

Candidates: $y_i,y_j$

Pair-specific scores: $s^i_{(i,j)}, s^j_{(i,j)}$

Confidence that $y_i$ is better than $y_j$
$$s_{ij} = s^i_{(i,j)} - s^j_{(i,j)}$$

### Methodology
Methods like RLHF/InstructGPT scored the prompt + completion (reward). PairRM introduces a different approach. They directly model the confidence that one sample is better than another.

![](https://i.ibb.co/xJG6fcS/Screenshot-2024-01-25-at-10-23-46-PM.png)




The PairRanker is a BERT-structure encoder, and is fine-tuned on DeBERTa. It encodes the input text with two candidates using a single encoders and ouptuts two predictions scores for the the two candidates. The better candidates is expected to get a high score. Authors suggest that the bi-directional attention mechanism may make it more sensitive to determining the quality of the candidates.

$$L_Q = -z_i \log(\sigma(Q(s_i^i))) - z_j \log(\sigma(Q(s_i^j)))$$

here $\sigma$ denotes the sigmoid function and
$$(z_i, z_j) =
\begin{cases}
  (1,0), & \text{if } Q(y_i) \geq Q(y_j) \\
  (0,1), & \text{if } Q(y_i) < Q(y_j)
\end{cases}$$


Here Q, represents the quality as measure by a number of different quality metrics:

- BERTScore: Measures semantic similarity between generated and reference texts using BERT embeddings and cosine similarity.
- ROUGE: Assesses overlap of N-grams between generated and reference texts, used mainly for summarization and translation.
- BLEU: Calculates precision of N-grams in machine translations with a penalty for brevity, focusing on word overlap.


### Concrete Example

Let's work out the examples for when the model predicts correctly ($z_i = 0.9$) and when it does not predict correctly ($z_i = 0.1$). We'll use the given loss function:

$$L_Q = -z_i \log(\sigma(Q(s^j))) - z_j \log(\sigma(Q(s^j)))$$

#### Correct: Ground truth ($z_i = 1.0$, $z_j = 0.0$) Prediction ($z_i = 0.9$, $z_j = 0.1$)

In this case, we assume that $Q(y_i) \geq Q(y_j)$, so $(z_i, z_j) = (0.9, 0.1)$. Let's also assume that $Q(s_i^j) = 0.9$, which means the model's prediction is close to the true label.

The loss function becomes:

$$L_Q = -1\log(\sigma(0.9)) - 0 \log(\sigma(0.9))$$

Using the sigmoid function $\sigma(x) = \frac{1}{1 + e^{-x}}$, we have:

$$\sigma(0.9) \approx 0.71$$

So the loss is:

$$L_Q = -0.9 \log(0.71) - 0.1 \log(0.71) \approx 0.9 \times 0.34 + 0.1 \times 0.34 = 0.34$$

#### Incorrect: Ground truth ($z_i = 1.0$, $z_j = 0.0$) Prediction ($z_i = 0.1$, $z_j = 0.9$)

In this case, we assume that $Q(y_i) < Q(y_j)$, so $(z_i, z_j) = (0.1, 0.9)$. Let's also assume that $Q(s_i^j) = -0.9$, which means the model's prediction is far from the true label.

The loss function becomes:

$$L_Q = -0.1 \log(\sigma(-0.9)) - 0.9 \log(\sigma(-0.9))$$

Using the sigmoid function, we have:

$$\sigma(-0.9) \approx 0.29$$

So the loss is:

$$L_Q = -0.1 \log(0.29) - 0.9 \log(0.29) \approx 0.1 \times 1.23 + 0.9 \times 1.23 = 1.23$$

In summary, when the model predicts correctly, the loss is lower ($0.34$), and when the model predicts incorrectly, the loss is higher ($1.23$). This demonstrates that the loss function effectively penalizes incorrect predictions more than correct ones.


In [ ]:
!pip install git+https://github.com/yuchenlin/LLM-Blender.git

  Cloning https://github.com/yuchenlin/LLM-Blender.git to /tmp/pip-req-build-apmpk6p_
  Running command git clone --filter=blob:none --quiet https://github.com/yuchenlin/LLM-Blender.git /tmp/pip-req-build-apmpk6p_
  Resolved https://github.com/yuchenlin/LLM-Blender.git to commit 3c2d71f4698af48d09a50f67ad97937a73122599
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 27.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━

In [ ]:
import llm_blender
blender = llm_blender.Blender()
blender.loadranker("llm-blender/PairRM")

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/130 [00:00<?, ?B/s]

...

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/874M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Successfully loaded ranker from  /root/.cache/huggingface/hub/llm-blender/PairRM


In [ ]:
inputs = ["hello, how are you!", "I love you!"]
candidates_texts = [["get out!", "hi! I am fine, thanks!", "bye!"],
                    ["I love you too!", "I hate you!", "Thanks! You're a good guy!"]]
ranks = blender.rank(inputs, candidates_texts, return_scores=False, batch_size=1)
# ranks is a list of ranks where ranks[i][j] represents the ranks of candidate-j for input-i
"""
ranks -->
array([[3, 1, 2], # it means "hi! I am fine, thanks!" ranks the 1st, "bye" ranks the 2nd, and "get out!" ranks the 3rd.
       [1, 3, 2]], # it means "I love you too"! ranks the the 1st, and "I hate you!" ranks the 3rd.
       dtype=int32)

"""

Ranking candidates: 100%|██████████| 2/2 [00:02<00:00,  1.18s/it]


'\nranks -->\narray([[3, 1, 2], # it means "hi! I am fine, thanks!" ranks the 1st, "bye" ranks the 2nd, and "get out!" ranks the 3rd. \n       [1, 3, 2]], # it means "I love you too"! ranks the the 1st, and "I hate you!" ranks the 3rd.\n       dtype=int32) \n\n'

In [ ]:
print(ranks)

[[3 1 2]
 [1 3 2]]


# Self Rewarding LMs (Iterative DPO)
https://arxiv.org/pdf/2401.10020.pdf

"to achieve superhuman agents, future models require super-human feedback"


In RLHF, the reward model is typically frozen.

In RLHF/DPO, human preference data quality and size is the bottleneck.

They train a self-improving reward model that, rather than being frozen, is continually updated duing LLM alignment. They key is to develop an agent that possesses all the abilities desired during training, rahter than separating them out into distinct models such as a reward model and a language model. In the same way that pretraining and multitasking training of instruction following tasks allow task transfer by training on many tasks at once, incorporating the reward model into the same system allows task transfer between the reward modeling task and the instruction following tasks.

![](https://i.ibb.co/W2CFMkW/Screenshot-2024-01-25-at-10-13-36-PM.png)

Self-Rewarding LM can
1. act as instruction following models
2. can generate and evaluate new instruction following examples to add to their own training set



## Metholodogy Overview

1. Self Instruction Creation: newly created prompts are used to generated candidate responses from model $M_t$, which also predicts its own rewards via LLM-as-a-Judge prompting
2. Instruction following training: Preference pairs are selected from the generated data, which are used for training via DPO, resulting in model $M_{t+1}$.
3. Repeat



## Data

### Seed Data

#### Instruction Data

Instruction Following Data (IFT): sampled 3200 human written instruction-response pairs from OpenAssistant, using only the first conversation turns that are scored the highest (annotated by humans)


#### LLM-as-a-Judge Data
* LLM-as-a-Judge Instruction Following Data (EFT): used ranked human responses for each instruction from OpenAssistant dataset
    
* evaluation instruction + evaluation response pairs
* LM(CoT(evaluation instruction)) -> Reasoning + Score

```
Review the user’s question and the corresponding response using the additive 5-point
scoring system described below. Points are accumulated based on the satisfaction of
each criterion:
- Add 1 point if the response is relevant and provides some information related to
the user’s inquiry, even if it is incomplete or contains some irrelevant content.
- Add another point if the response addresses a substantial portion of the user’s
question, but does not completely resolve the query or provide a direct answer.
- Award a third point if the response answers the basic elements of the user’s question
in a useful way, regardless of whether it seems to have been written by an AI Assistant
or if it has elements typically found in blogs or search results.
- Grant a fourth point if the response is clearly written from an AI Assistant’s
perspective, addressing the user’s question directly and comprehensively, and is
well-organized and helpful, even if there is slight room for improvement in clarity,
conciseness or focus.
- Bestow a fifth point for a response that is impeccably tailored to the user’s question
by an AI Assistant, without extraneous information, reflecting expert knowledge, and
demonstrating a high-quality, engaging, and insightful answer.

User: <INSTRUCTION_HERE>
<response><RESPONSE_HERE></response>

After examining the user’s instruction and the response:
- Briefly justify your total score, up to 100 words.
- Conclude with the score using the format: “Score: <total points>”

Remember to assess from the AI Assistant perspective, utilizing web search knowledge as
necessary. To evaluate the response in alignment with this additive scoring model, we’ll
systematically attribute points based on the outlined criteria

```

### Synthetic Data Generation
#### Instruction-Response Pairs
* Instruction: Generate new instructions via 8-hot prompting with instructions from the original IFT data
* Responses: Generate 3 candidate responses with temperature = 0.7 and top P = 0.9.
* Response Scores: Use LLM-as-a-Judge Prompt to score candidate responses on a scale of 0-5. Each response is scored 3 times and score is averaged

They create preference pair data: (instruction $x_i$, winning response $y_i^w$, losing response $y_i^l$). This is the AIFT dataset








## Training

They iteratively finetuned a series of models $M_1 \ldots M_T$ where each successive model $t$ uses augmented training data generated by the $t - 1^{th}$ model:

- $M_0$: Base pretrained LLM with no fine-tuning
- $M_1$: Initialized with $M_0$, then finetuned on the IFT + EFT seed data via SFT
- $M_2$: Initialized with $M_1$, then trained with $AIFT(M_1)$ data using DPO
- $M_3$: Initialized with $M_2$, then trained with $AIFT(M_2)$ data using DPO




## Results

For instruction-following, each iteration improved over the previous. The SFT baseline is surpassed by $M_2$ (iteration 2). The improvements from iterations $M_1$ to $M_2$ to $M_3$ didn’t seem to taper much so there may be further improvements from additional iterations.

![](https://eugeneyan.com/assets/self-reward-fig3.jpg)

 $M_3$ (iteration 3) outperformed several existing models that use proprietary data (e.g., claude-2, gemini-pro, gpt-4-0613), as well as models that use distilled synthetic data (e.g., Alpaca, Vicuna).

 ![](https://eugeneyan.com/assets/self-reward-table1.jpg)
